In [0]:
!wget wget -O /dbfs/orders.csv https://s3-geospatial.s3.us-west-2.amazonaws.com/orders.csv

--2025-03-25 21:12:51--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2025-03-25 21:12:51--  https://s3-geospatial.s3.us-west-2.amazonaws.com/orders.csv
Resolving s3-geospatial.s3.us-west-2.amazonaws.com (s3-geospatial.s3.us-west-2.amazonaws.com)... 52.92.238.42, 3.5.77.252, 52.92.152.82, ...
Connecting to s3-geospatial.s3.us-west-2.amazonaws.com (s3-geospatial.s3.us-west-2.amazonaws.com)|52.92.238.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89951 (88K) [text/csv]
Saving to: ‘/dbfs/orders.csv’

/dbfs/orders.csv    100%[===================>]  87.84K  --.-KB/s    in 0.001s  

2025-03-25 21:12:52 (79.6 MB/s) - ‘/dbfs/orders.csv’ saved [89951/89951]

FINISHED --2025-03-25 21:12:52--
Total wall clock time: 0.2s
Downloaded: 1 files, 88K in 0.001s (79.6 MB/s)


In [0]:
spark.conf.get("spark.sql.catalogImplementation")

'hive'

In [0]:
spark.sql("SHOW CATALOGS").show()

+-------------+
|      catalog|
+-------------+
|spark_catalog|
+-------------+



In [0]:
spark.conf.get("spark.sql.warehouse.dir")

'dbfs:/user/hive/warehouse'

In [0]:
spark.sql("SHOW DATABASES").show()

+------------+
|databaseName|
+------------+
|     default|
+------------+



In [0]:
spark.sql("SELECT CURRENT_DATABASE()").show()

+----------------+
|current_schema()|
+----------------+
|         default|
+----------------+



In [0]:
# 새로 데이터베이스를 하나 만들고 그걸 사용
spark.sql("CREATE DATABASE IF NOT EXISTS DEV")
spark.sql("USE DEV")

DataFrame[]

In [0]:
spark.sql("SHOW DATABASES").show()

+------------+
|databaseName|
+------------+
|     default|
|         dev|
+------------+



In [0]:
spark.sql("SELECT CURRENT_DATABASE()").show()

+----------------+
|current_schema()|
+----------------+
|             dev|
+----------------+



In [0]:
%fs cp file:/dbfs/orders.csv dbfs:/orders.csv

res21: Boolean = true

In [0]:
# 위 파일은 탭 문자를 분리자로 사용 중임
df = spark.read.csv("dbfs:/orders.csv", inferSchema=True, header=True, sep ='\t')

In [0]:
df.show()

+------------+--------------------+
|    order_id|               items|
+------------+--------------------+
|860196503764|[{"name": "DAILY ...|
|860292645076|[{"name": "DAILY ...|
|860320956628|[{"name": "DAILY ...|
|860321513684|[{"name": "DAILY ...|
|862930665684|[{"name": "DAILY ...|
|862975819988|[{"name": "DAILY ...|
|862985191636|[{"name": "DAILY ...|
|870939295956|[{"name": "DAILY ...|
|880188063956|[{"name": "DAILY ...|
|933014601940|[{"name": "DAILY ...|
|934065930452|[{"name": "DAILY ...|
|938210722004|[{"name": "DAILY ...|
|944748331220|[{"name": "DAILY ...|
|862843896020|[{"name": "DAILY ...|
|862959763668|[{"name": "DAILY ...|
|870966558932|[{"name": "DAILY ...|
|887936647380|[{"name": "DAILY ...|
|908426477780|[{"name": "DAILY ...|
|921300107476|[{"name": "DAILY ...|
|932229710036|[{"name": "DAILY ...|
+------------+--------------------+
only showing top 20 rows



In [0]:
df.write.saveAsTable("DEV.orders", mode="overwrite")

In [0]:
spark.sql("SHOW TABLES").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
|     dev|   orders|      false|
+--------+---------+-----------+



In [0]:
spark.catalog.listTables()

[Table(name='orders', catalog='spark_catalog', namespace=['dev'], description=None, tableType='MANAGED', isTemporary=False)]

In [0]:
%fs ls dbfs:/user/hive/warehouse

path,name,size,modificationTime
dbfs:/user/hive/warehouse/dev.db/,dev.db/,0,0


In [0]:
%fs ls dbfs:/user/hive/warehouse/dev.db/

path,name,size,modificationTime
dbfs:/user/hive/warehouse/dev.db/orders/,orders/,0,0


In [0]:
%fs ls dbfs:/user/hive/warehouse/dev.db/orders/

path,name,size,modificationTime
dbfs:/user/hive/warehouse/dev.db/orders/_delta_log/,_delta_log/,0,0
dbfs:/user/hive/warehouse/dev.db/orders/part-00000-381c7abf-a38d-4d0f-931c-02b7a032089b-c000.snappy.parquet,part-00000-381c7abf-a38d-4d0f-931c-02b7a032089b-c000.snappy.parquet,23224,1742937633000


In [0]:
spark.table("DEV.orders").show(10)

+------------+--------------------+
|    order_id|               items|
+------------+--------------------+
|860196503764|[{"name": "DAILY ...|
|860292645076|[{"name": "DAILY ...|
|860320956628|[{"name": "DAILY ...|
|860321513684|[{"name": "DAILY ...|
|862930665684|[{"name": "DAILY ...|
|862975819988|[{"name": "DAILY ...|
|862985191636|[{"name": "DAILY ...|
|870939295956|[{"name": "DAILY ...|
|880188063956|[{"name": "DAILY ...|
|933014601940|[{"name": "DAILY ...|
+------------+--------------------+
only showing top 10 rows



In [0]:
spark.sql("SELECT * FROM DEV.orders LIMIT 10").show()

+------------+--------------------+
|    order_id|               items|
+------------+--------------------+
|860196503764|[{"name": "DAILY ...|
|860292645076|[{"name": "DAILY ...|
|860320956628|[{"name": "DAILY ...|
|860321513684|[{"name": "DAILY ...|
|862930665684|[{"name": "DAILY ...|
|862975819988|[{"name": "DAILY ...|
|862985191636|[{"name": "DAILY ...|
|870939295956|[{"name": "DAILY ...|
|880188063956|[{"name": "DAILY ...|
|933014601940|[{"name": "DAILY ...|
+------------+--------------------+



In [0]:
spark.sql("""
    CREATE TABLE DEV.orders_count AS
    SELECT order_id, COUNT(1) as count
    FROM DEV.orders
    GROUP BY 1""")

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
# external table을 만들어볼 예정 - 디렉토리 지정이 필요함!
spark.sql("""
    CREATE EXTERNAL TABLE DEV.ext_orders (
        order_id LONG,
        items STRING
    )
    ROW FORMAT DELIMITED
    FIELDS TERMINATED BY '\t'
    STORED AS TEXTFILE
    LOCATION 'dbfs:/orders.csv'
""")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2313936765186963>, line 2
      1 # external table을 만들어볼 예정 - 디렉토리 지정이 필요함!
----> 2 spark.sql("""
      3     CREATE EXTERNAL TABLE DEV.ext_orders (
      4         order_id LONG,
      5         items STRING
      6     )
      7     ROW FORMAT DELIMITED
      8     FIELDS TERMINATED BY '\t'
      9     STORED AS TEXTFILE
     10     LOCATION 'dbfs:/orders.csv'
     11 """)

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/session.py:1825, in SparkSession.sql(sel

In [0]:
%fs mkdirs dbfs:/orders/

res25: Boolean = true

In [0]:
%fs cp dbfs:/orders.csv dbfs:/orders/

res26: Boolean = true

In [0]:
%sql

CREATE EXTERNAL TABLE DEV.ext_orders (
  order_id LONG,
  items STRING
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t'
STORED AS TEXTFILE
LOCATION 'dbfs:/orders/'

In [0]:
spark.sql("SHOW TABLES").show()

+--------+------------+-----------+
|database|   tableName|isTemporary|
+--------+------------+-----------+
|     dev|  ext_orders|      false|
|     dev|      orders|      false|
|     dev|orders_count|      false|
+--------+------------+-----------+



In [0]:
# 결과에서 tableType을 살펴볼 것!
spark.catalog.listTables()

[Table(name='ext_orders', catalog='spark_catalog', namespace=['dev'], description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='orders', catalog='spark_catalog', namespace=['dev'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='orders_count', catalog='spark_catalog', namespace=['dev'], description=None, tableType='MANAGED', isTemporary=False)]

In [0]:
spark.sql("SELECT * FROM DEV.ext_orders").show()

+------------+--------------------+
|    order_id|               items|
+------------+--------------------+
|        NULL|               items|
|860196503764|[{"name": "DAILY ...|
|860292645076|[{"name": "DAILY ...|
|860320956628|[{"name": "DAILY ...|
|860321513684|[{"name": "DAILY ...|
|862930665684|[{"name": "DAILY ...|
|862975819988|[{"name": "DAILY ...|
|862985191636|[{"name": "DAILY ...|
|870939295956|[{"name": "DAILY ...|
|880188063956|[{"name": "DAILY ...|
|933014601940|[{"name": "DAILY ...|
|934065930452|[{"name": "DAILY ...|
|938210722004|[{"name": "DAILY ...|
|944748331220|[{"name": "DAILY ...|
|862843896020|[{"name": "DAILY ...|
|862959763668|[{"name": "DAILY ...|
|870966558932|[{"name": "DAILY ...|
|887936647380|[{"name": "DAILY ...|
|908426477780|[{"name": "DAILY ...|
|921300107476|[{"name": "DAILY ...|
+------------+--------------------+
only showing top 20 rows



In [0]:
%sql

SELECT *
FROM dev.ext_orders
LIMIT 10;

order_id,items
null,items
860196503764,"[{""name"": ""DAILY SPF"", ""quantity"": 1, ""id"": 1883727790094}]"
860292645076,"[{""name"": ""DAILY SPF \u2014 Bundle Set"", ""quantity"": 1, ""id"": 1883875377166}]"
860320956628,"[{""name"": ""DAILY SPF"", ""quantity"": 1, ""id"": 1883919974414}]"
860321513684,"[{""name"": ""DAILY SPF"", ""quantity"": 1, ""id"": 1883920793614}]"
862930665684,"[{""name"": ""DAILY SPF"", ""quantity"": 1, ""id"": 1887913672718}]"
862975819988,"[{""name"": ""DAILY SPF"", ""quantity"": 1, ""id"": 1887985827854}]"
862985191636,"[{""name"": ""DAILY SPF \u2014 Bundle Set"", ""quantity"": 1, ""id"": 1887999164430}]"
870939295956,"[{""name"": ""DAILY SPF"", ""quantity"": 1, ""id"": 1900142264334}]"
880188063956,"[{""name"": ""DAILY SPF"", ""quantity"": 1, ""id"": 1914170572814}]"


In [0]:
# external table의 삭제는 실제 파일의 삭제로 이어지지 않는다는 점 확인
spark.sql("DROP TABLE IF EXISTS DEV.ext_orders")

DataFrame[]

In [0]:
%fs ls dbfs:/orders/

path,name,size,modificationTime
dbfs:/orders/orders.csv,orders.csv,89951,1742937796000


In [0]:
spark.sql("DROP TABLE IF EXISTS DEV.orders")

DataFrame[]

In [0]:
%fs ls dbfs:/user/hive/warehouse/dev.db/

path,name,size,modificationTime
dbfs:/user/hive/warehouse/dev.db/orders_count/,orders_count/,0,0
